In [11]:
import geopandas as gpd
from shapely.wkt import loads
from shapely.wkt import dumps

# Assuming you have a WKT string
wkt_string = "LINESTRING (-17 51, 20 40, 30 60)"

# Convert the WKT string to a Shapely geometry object
geometry = loads(wkt_string)
print(dumps(geometry))
print(type(geometry))
# Create a GeoSeries from the Shapely geometry object
geo_series = gpd.GeoSeries(geometry)

# Now you have a GeoSeries containing the geometry
print(geo_series)
print(geo_series[0])


LINESTRING (-17.0000000000000000 51.0000000000000000, 20.0000000000000000 40.0000000000000000, 30.0000000000000000 60.0000000000000000)
<class 'shapely.geometry.linestring.LineString'>
0    LINESTRING (-17.00000 51.00000, 20.00000 40.00...
dtype: geometry
LINESTRING (-17 51, 20 40, 30 60)


In [17]:
from collections import OrderedDict
from flask import jsonify
import json
# Create an ordered dictionary to maintain the order of keys
response_data = OrderedDict([
    ('locationID', 4),
    ('decimalLongitude', 5),
    ('coordinatePrecision', 0.0000001)
])
response_data
# Convert the ordered dictionary to JSON response
json.dumps(response_data)

'{"locationID": 4, "decimalLongitude": 5, "coordinatePrecision": 1e-07}'

In [21]:
# Simple check to assume coordinates are in EPSG4326
def isLatLon(lon, lat):
  ok = True
  if lon > 180 or lon < -180:
    ok = False
  if lat > 90 or lat < -90:
    ok = False
  return ok  

isLatLon(45, 7)

True

In [34]:
from flask import Flask, request, jsonify
# json.dumps(OrderedDict([('Error', "Geometry does not appear to be in EPSG:4326 (Lat/Lon)")]))
# json.dumps({'Error', 'Geometry does not appear to be in EPSG:4326 (Lat/Lon)'})
# error_data = {'Error': "Geometry does not appear to be in EPSG:4326 (Lat/Lon)"}
json.dumps({"Error": "Geometry does not appear to be in EPSG:4326 (Lat/Lon)"})
# Convert the dictionary to a JSON string
# json_response = json.dumps(error_data)
# json_response

'{"Error": "Geometry does not appear to be in EPSG:4326 (Lat/Lon)"}'

In [55]:
from shapely.geometry import Polygon, MultiPolygon, LineString, MultiLineString


def isLatLon(lat, lon): 
  ok = True
  if lon > 180 or lon < -180:
    ok = False
  if lat > 90 or lat < -90:
    ok = False
  return ok  

def iterate_coordinates(geometry):
    if geometry.geom_type == 'Polygon':
        for point in geometry.exterior.coords:
            yield point
    elif geometry.geom_type == 'MultiPolygon':
        for polygon in geometry.geoms:
            for point in polygon.exterior.coords:
                yield point
    elif geometry.geom_type == 'LineString':
        for point in geometry.coords:
            yield point
    elif geometry.geom_type == 'MultiLineString':
        for linestring in geometry.geoms:
            for point in linestring.coords:
                yield point

def wktIsLatLon(wkt):
  wktOK = True
  for point in iterate_coordinates(wkt): 
      if not isLatLon(point[1], point[0]):
        wktOK = False
        break
  return wktOK

# Example geometries
polygon = Polygon([(0, 0), (10, 10), (1, 1), (0, 1), (0, 0)])
multi_polygon = MultiPolygon([polygon])
line_string = LineString([(0, 0), (1, 1), (200, 0)])
multi_line_string = MultiLineString([line_string])

wktIsLatLon(polygon)


All points ok


In [56]:
import geopandas as gpd
s = '{"type": "Feature", "properties": {}, "geometry": {"type": "LineString", "coordinates": [[[-50.29953, 58.385518], [-26.115189, 65.000261], [-44.378586, 41.740066]]]}}'
gpd.read_file(s, driver = 'GeoJSON')

,geometry
0,None


In [57]:
json.dumps(("Error", "Geometry does not appear to be in EPSG:4326 (Lat/Lon)"))

'["Error", "Geometry does not appear to be in EPSG:4326 (Lat/Lon)"]'

In [14]:
import geopandas as gpd
from shapely.geometry import shape

# Define your JSON data
json_data = {
  "type": "Feature",
  "properties": {},
  "geometry": {
    "type": "MultiPolygon",
    "coordinates": [
      [
        [
          [0, 0],
          [10, 0],
          [10, 10],
          [0, 10],
          [0, 0]
        ],
        [
          [2, 2],
          [2, 8],
          [8, 8],
          [8, 2],
          [2, 2]
        ]
      ],
      [
        [
          [20, 20],
          [30, 20],
          [30, 30],
          [20, 30],
          [20, 20]
        ],
        [
          [22, 22],
          [22, 28],
          [28, 28],
          [28, 22],
          [22, 22]
        ]
      ]
    ]
  }
}


# Convert JSON data to a GeoDataFrame
geometry = shape(json_data['geometry'])
gdf = gpd.GeoDataFrame(geometry=[geometry])

# Convert GeoDataFrame to GeoSeries
geoseries = gdf['geometry']

print(geoseries[0])

MULTIPOLYGON (((0 0, 10 0, 10 10, 0 10, 0 0), (2 2, 2 8, 8 8, 8 2, 2 2)), ((20 20, 30 20, 30 30, 20 30, 20 20), (22 22, 22 28, 28 28, 28 22, 22 22)))


,x,y
0,0.0,0.0
0,10.0,0.0
0,10.0,10.0
0,0.0,10.0
0,0.0,0.0
0,2.0,2.0
0,2.0,8.0
0,8.0,8.0
0,8.0,2.0
0,2.0,2.0


In [15]:
geoseries.to_json()

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {}, "geometry": {"type": "MultiPolygon", "coordinates": [[[[0.0, 0.0], [10.0, 0.0], [10.0, 10.0], [0.0, 10.0], [0.0, 0.0]], [[2.0, 2.0], [2.0, 8.0], [8.0, 8.0], [8.0, 2.0], [2.0, 2.0]]], [[[20.0, 20.0], [30.0, 20.0], [30.0, 30.0], [20.0, 30.0], [20.0, 20.0]], [[22.0, 22.0], [22.0, 28.0], [28.0, 28.0], [28.0, 22.0], [22.0, 22.0]]]]}, "bbox": [0.0, 0.0, 30.0, 30.0]}], "bbox": [0.0, 0.0, 30.0, 30.0]}'

In [32]:
coordinates_df = geoseries.get_coordinates()
for index, row in coordinates_df.iterrows():
    # Access row data using row['column_name']
    print(row['x'])

0.0
10.0
10.0
0.0
0.0
2.0
2.0
8.0
8.0
2.0
20.0
30.0
30.0
20.0
20.0
22.0
22.0
28.0
28.0
22.0
